In [ ]:
# Train SVM with polynomial kernel
# Create 327x3 matrix, for each image set contains neutral image name, expression image name, emotion label
# Generate normalized Euclidean Differences array
# Generate target values array
# Feed to SVM, evaulate with Cross Validation



##### SET UP METHODS #####
import numpy as np
import cv2
import dlib
import os
import math
from sklearn import svm
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    for d in dets:
        shape = predictor(image, d)
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)

##### MAIN PROGRAM STARTS #####
# Get paths to face images and emotion labels
imagedir = []
labeldir = []
root, dirs, files = os.walk("./cohn-kanade-images2",).__next__()
for d in dirs:
    imagedir.append(os.path.join(root,d))
    labeldir.append(os.path.join('./Emotion',d))

# initialize variables
dataset = []
NLA = []
ELA = []

# look over all directories of individuals S001 through S999
for d in dirs:
    for dd in os.listdir(os.path.join('./cohn-kanade-images2',d)):
        # skip over .DS_Store files
        if 'Store' not in dd:
            curdir = os.path.join('./cohn-kanade-images2',d,dd)
            files = os.listdir(curdir)
            baseimage = os.path.join(curdir,files[0])
            faceimage = os.path.join(curdir,files[-1])

            # get directories of multiple expressions of same person
            labeldir = os.path.join('./Emotion',d,dd)
            if os.path.isdir(labeldir):
                labels = os.listdir(labeldir)
                # read label file if it exists
                if len(labels)>0:
                    f=open(os.path.join(labeldir,labels[0]),"r")
                    for line in f:
                        targetlabel = float(line)
                    f.close()
                    
                    # append label to dataset
                    if int(targetlabel)!=2:
                        dataset.append([baseimage, faceimage, targetlabel])

# Get array of Euclidean differences between neutral and expression 68-landmark arrays
tempLDA = []
for imgIndex in range(len(dataset)):
    nImgName = dataset[imgIndex][0]
    eImgName = dataset[imgIndex][1]
    nLA = get68LMA(nImgName)
    eLA = get68LMA(eImgName)
    NLA.append(nLA)
    ELA.append(eLA)
    tempLDA.append(getEucDist(nLA, eLA))
LDA = np.asmatrix(tempLDA)

# Create target value array for expression images
tempTargetValues = []
for imgSet in range(len(dataset)):
    tempTargetValues.append(dataset[imgSet][2])
targetValues = np.asarray(tempTargetValues)

##### TRAIN PHYSIOLOGICAL MODEL #####
# Get array of corresponding differences between 13 distances of base images and expression images
tempPDA = []
for i in range(len(NLA)):
    tempPDA.append(getPDA(NLA[i])-getPDA(ELA[i]))
PDA = np.asmatrix(tempPDA)

# Feed PDA and targetValues into SVM, use cross validation, generate confusion matrix
x_train2, x_test2, y_train2, y_test2 = cross_validation.train_test_split(PDA, targetValues, test_size=0.4, random_state=3)
physCLF = svm.SVC(kernel='poly').fit(x_train2, y_train2)
predictions = physCLF.predict(x_test2)
cMatrix = confusion_matrix(y_test2, predictions)
print(cMatrix)

In [3]:
# Capture video
# Classify emotion
# Output the emotion

import cv2
import dlib
import numpy as np
from sklearn.externals import joblib

cap = cv2.VideoCapture(0) # capture video from robot camera
cap.set(3,640)
cap.set(4,480)
loaded = joblib.load('theModel.pkl')
detector = dlib.get_frontal_face_detector()

images=[]

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    dets = detector(frame, 1)
    
    for rect in dets:
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    cv2.imshow('frame',frame)

    # Get physiological feature vector for current frame
    images.append(getPhysDA(getI68LMA(frame)))
    
    if len(images)>2:
        images.pop(0)
    # Classify expression in image
    if len(images)>1:
        index=loaded.predict(np.asarray(images[0]-images[1])) # Get array of probabilities
        print(index)
        
        # Print the emotion corresponding to the position
        if index==1:
            print("Anger")
        elif index==3:
            print("Disgust")
        elif index==4:
            print("Fear")
        elif index==5:
            print("Happiness")
        elif index==6:
            print("Sadness")
        elif index==7:
            print("Surprise")
            
    # Break out of while loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break      
        
# Release video capture
cap.release()
cv2.destroyAllWindows()

NameError: name 'getPhysDA' is not defined

In [ ]:
# Capture video
# Classify emotion
# Output the emotion

import cv2
import dlib
import numpy as np
import os
import math
from sklearn.externals import joblib

# Function that returns the distance between two given points
def ptDist(pt1, pt2):
    x1 = pt1[0]
    x2 = pt2[0]
    y1 = pt1[1]
    y2 = pt2[1]
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

# Function that returns array of 68 landmarks for given image name
def get68LMA(imgName):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    image = cv2.imread(imgName, 1)
    
    # Detect face(s)
    dets = detector(image, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    shape = predictor(image, dets[0])
    for index in range(0, 68):
        point = shape.part(index)
        tuplepoint = (point.x, point.y)
        LMA.append(tuplepoint)
    return LMA

# Function that eturns normalized array of Euclidean distances given two arrays of coordinates
def getEucDist(nLA, eLA):
    LDA = []
    sum = 0
    # Get array of distances
    for pt in range(68):
        dif = ptDist(eLA[pt], nLA[pt])     
        LDA.append(dif)
        sum += dif
    avg = sum/68
    # Normalize array
    for i in range(68):
        LDA[i] -= avg      
    return LDA

# Method to get 68 landmarks for IMAGE passed to the function
def getI68LMA(img):
    predictor_path = "/Users/kaili/Dropbox/HSSF1516/shape_predictor_68_face_landmarks.dat"
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(predictor_path)
    LMA = []
    
    # Detect face(s)
    dets = detector(img, 1)

    # Get list of coordinates of 68 facial landmarks for each face
    if len(dets)>0:
        shape = predictor(img, dets[0])
        for index in range(0, 68):
            point = shape.part(index)
            tuplepoint = (point.x, point.y)
            LMA.append(tuplepoint)
        return LMA

# Function that returns 13-point numpy vector of distances between selected physiological pts, given the 68-point vector
def getPhysDA(iLA):
    PDA = []
    PDA.append(ptDist(iLA[20], iLA[21])) #distance 0: distance between brows
    PDA.append(ptDist(iLA[20], iLA[26])) #distance 1: inner brow to nose- left
    PDA.append(ptDist(iLA[21], iLA[26])) #distance 2: inner brow to nose- right
    PDA.append(ptDist(iLA[20], iLA[38])) #distance 3: inner brow to inner eye corner- left
    PDA.append(ptDist(iLA[21], iLA[41])) #distance 4: inner brow to inner eye corner- right
    PDA.append(ptDist(iLA[35], iLA[47])) #distance 5: outer eye corner to outer mouth corner- left
    PDA.append(ptDist(iLA[44], iLA[53])) #distance 6: outer eye corner to outer mouth corner- right
    PDA.append(ptDist(iLA[47], iLA[53])) #distance 7: distance between mouth corners
    PDA.append(ptDist(iLA[61], iLA[65])) #distance 8: height of mouth
    PDA.append(ptDist(iLA[35], iLA[49])) #distance 9: raising of upper lip
    PDA.append(ptDist(iLA[44], iLA[51])) #distance 10: raising of upper lip
    PDA.append(ptDist(iLA[18], iLA[36])) #distance 11: brow arch- left
    PDA.append(ptDist(iLA[23], iLA[43])) #distance 12: brow arch- right
    return np.asarray(PDA)

    
cap = cv2.VideoCapture(0) # capture video from robot camera
cap.set(3,640)
cap.set(4,480)
loaded = joblib.load('poly7.pkl')
detector = dlib.get_frontal_face_detector()

images=[]


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    dets = detector(frame, 1)
    
    for rect in dets:
        cv2.rectangle(frame, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
    cv2.imshow('frame',frame)

    # Get physiological feature vector for current frame
    images.append(getPhysDA(getI68LMA(frame)))
    
    if len(images)>2:
        images.pop(0)
    # Classify expression in image
    if len(images)>1:
        emo_probs = loaded.predict_proba(np.asarray(images[0]-images[1])) # Get array of probabilities
        index = np.argmax(emo_probs)# Get position of highest probability
#         if magnitude > 0.8:
#             print("Strong")
#         elif magnitude > 0.4:
#             print("Moderate")
        
        # Print the emotion corresponding to the position
        if index==0 or index==6:
            print("Anger")
            #os.system("ssh pi@192.168.1.108 python emotions/anger.py")
        elif index==1 or index==7:
            print("Disgust")
        elif index==2 or index==8:
            print("Fear")
        elif index==3 or index==9:
            print("Happiness")
            #os.system("ssh pi@192.168.1.108 python emotions/happiness.py")
        elif index==4 or index==10:
            print("Sadness")
            #os.system("ssh pi@192.168.1.108 python emotions/sadness.py")
        elif index==5 or index==11:
            print("Surprise")
            #os.system("ssh pi@192.168.1.108 python emotions/surprise.py")
            
    # Break out of while loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break      
        
# Release video capture
cap.release()
cv2.destroyAllWindows()

/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Surprise


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Surprise


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


...

Surprise


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Surprise


/Users/kaili/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Surprise
